In [1]:
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.algo.filtering.log.attributes import attributes_filter

import numpy as np
import pandas as pd

In [3]:
log = xes_import_factory.apply("data/DomesticDeclarations.xes")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use importer entrypoint instead
  """Entry point for launching an IPython kernel.


In [4]:
log

[{'attributes': {'Amount': 26.85120450862128, 'id': 'declaration 86791', 'BudgetNumber': 'budget 86566', 'DeclarationNumber': 'declaration number 86792', 'concept:name': 'declaration 86791'}, 'events': [{'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'st_step 86794_0', 'org:role': 'EMPLOYEE', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'org:resource': 'STAFF MEMBER'}, '..', {'time:timestamp': datetime.datetime(2017, 1, 12, 17, 31, 22, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'dd_declaration 86791_20', 'org:role': 'UNDEFINED', 'concept:name': 'Payment Handled', 'org:resource': 'SYSTEM'}]}, '....', {'attributes': {'Amount': 190.40457573396148, 'id': 'declaration 138359', 'BudgetNumber': 'budget 86566', 'DeclarationNumber': 'declaration number 138360', 'concept:name': 'declaration 138359'}, 'events': [{'time:timestamp': datetime.datetime(2018, 12, 29, 17, 50, 14, tzinfo=datetime.timezon

In [5]:
sequences = list()
for i in log:
    sequence = list()
    sequence.append('start')
    for j in i:
        sequence.append(j)
    sequence.append("end")
    
    sequences.append(sequence)
    
print(sequences[:1])

[['start', {'time:timestamp': datetime.datetime(2017, 1, 9, 9, 49, 50, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'st_step 86794_0', 'org:role': 'EMPLOYEE', 'concept:name': 'Declaration SUBMITTED by EMPLOYEE', 'org:resource': 'STAFF MEMBER'}, {'time:timestamp': datetime.datetime(2017, 1, 9, 11, 27, 48, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'st_step 86793_0', 'org:role': 'SUPERVISOR', 'concept:name': 'Declaration FINAL_APPROVED by SUPERVISOR', 'org:resource': 'STAFF MEMBER'}, {'time:timestamp': datetime.datetime(2017, 1, 10, 9, 34, 44, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'dd_declaration 86791_19', 'org:role': 'UNDEFINED', 'concept:name': 'Request Payment', 'org:resource': 'SYSTEM'}, {'time:timestamp': datetime.datetime(2017, 1, 12, 17, 31, 22, tzinfo=datetime.timezone(datetime.timedelta(0, 3600))), 'id': 'dd_declaration 86791_20', 'org:role': 'UNDEFINED', 'concept:name': 'Payment Handled', 'org:resource': 'SYSTEM'}, 'e

In [8]:
data = {'id': [], 'concept:name': [], 'org:role': [], 'org:resource': [], 'time:timestamp':  [], 'previous': [], 'next': []}
df_list = list()

for seq in sequences:
    dataframe = pd.DataFrame(data)
    # iterate through sequence and extract all previous and current fields
    for previous, current in zip(seq, seq[1:]):
        new_row = dict()
        if previous == "start" and current != "end":
            new_row['previous'] = "start"
        elif current != "end":
            new_row['previous'] = previous['concept:name']
        if current != "start" and current != "end":
            for field in current:
                new_row[field] = current[field]
        dataframe = dataframe.append(new_row, ignore_index=True)
    # iterate through sequence and extract all next values
    next_values = list()
    for curr, nex in zip(seq, seq[1:]):
        if curr != "end":
            if nex == "end":
                next_values.append(nex)
            else:
                next_values.append(nex['concept:name'])
    # add a NaN value to the end of the list, so that the second element is the first next element
    next_values.append(np.nan)
    dataframe['next'] = next_values[1:]
    # drop rows that only contain NaN
    dataframe = dataframe.dropna()
    df_list.append(dataframe)


In [9]:
all_df = pd.concat(df_list, ignore_index = True)
all_df[:10]

,concept:name,id,next,org:resource,org:role,previous,time:timestamp
0,Declaration SUBMITTED by EMPLOYEE,st_step 86794_0,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,EMPLOYEE,start,2017-01-09 09:49:50+01:00
1,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86793_0,Request Payment,STAFF MEMBER,SUPERVISOR,Declaration SUBMITTED by EMPLOYEE,2017-01-09 11:27:48+01:00
2,Request Payment,dd_declaration 86791_19,Payment Handled,SYSTEM,UNDEFINED,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-10 09:34:44+01:00
3,Payment Handled,dd_declaration 86791_20,end,SYSTEM,UNDEFINED,Request Payment,2017-01-12 17:31:22+01:00
4,Declaration SUBMITTED by EMPLOYEE,st_step 86798_0,Declaration APPROVED by PRE_APPROVER,STAFF MEMBER,EMPLOYEE,start,2017-01-09 10:26:14+01:00
5,Declaration APPROVED by PRE_APPROVER,st_step 86799_0,Declaration FINAL_APPROVED by SUPERVISOR,STAFF MEMBER,PRE_APPROVER,Declaration SUBMITTED by EMPLOYEE,2017-02-22 10:29:21+01:00
6,Declaration FINAL_APPROVED by SUPERVISOR,st_step 86797_0,Request Payment,STAFF MEMBER,SUPERVISOR,Declaration APPROVED by PRE_APPROVER,2017-02-23 08:14:45+01:00
7,Request Payment,dd_declaration 86795_19,Payment Handled,SYSTEM,UNDEFINED,Declaration FINAL_APPROVED by SUPERVISOR,2017-03-06 14:07:25+01:00
8,Payment Handled,dd_declaration 86795_20,end,SYSTEM,UNDEFINED,Request Payment,2017-03-13 17:30:59+01:00
9,Declaration SUBMITTED by EMPLOYEE,st_step 86804_0,Declaration APPROVED by PRE_APPROVER,STAFF MEMBER,EMPLOYEE,start,2017-01-09 11:13:33+01:00


In [ ]:
s = sequences[:1]
for i in s:
    print(i)
    for previous, current in zip(i, i[1:]):
        print("PREVIOUS ", previous, current)
    for current2, next2 in zip(i, i[1:]):
        print("NEXT ", current2, next2)

In [11]:
data = {'id': [], 'concept:name': [], 'org:role': [], 'org:resource': [], 'time:timestamp':  [], 'previous': [], 'next': []}
print(type(data))

<class 'dict'>
